In [12]:
#Import other useful libraries for segmenting and clustering
import pandas as pd
import numpy as np
import xml
import csv
import json
import requests
from bs4 import BeautifulSoup

In [13]:
#Get HTML data from wikipedia page
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text


In [14]:
#Apply beautiful soup to get url data and then prettify to get nested tags
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml') # make Soup using URL

In [15]:
#Parse the wiki data using table and tr tags based on the nested tagging done above
neighbourhood = soup.find('table', class_ = 'wikitable')
neighbourhood_rows = neighbourhood.find_all('tr')

In [16]:
#Get postal code, borough and neighbourhood from the table
information = []
for row in neighbourhood_rows:
    info = row.text.split('\n')[1:-1] # remove empty str (the first and last items)
    information.append(info)
    
#Convert to pandas data frame
neighbour_df = pd.DataFrame(information[1:], columns=information[0])

#Remove unassigned boroughs
neighbour_df = neighbour_df[neighbour_df.Borough != 'Not assigned']
neighbour_df.reset_index(drop=True, inplace=True)
neighbour_df.head(20)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


# Combining neighbourhoods with the same postal code

In [11]:
#Combining neighbourhoods with the same postal code
grouped = neighbour_df.groupby(['Postcode']) # group by Postcode
# combine the neighbourhoods grouped by postcode and into a new df
neighbourhood_grouped = grouped['Neighbourhood'].apply(lambda x: x.sum()) 
# adds spaces and commas between neighbourhoods
neighbourhood_grouped = grouped['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
# matches a borough to each postcode
borough_grouped = grouped['Borough'].apply(lambda x: set(x).pop())
# turn borough_grouped and neighbourhood_grouped into dataframes
borough = borough_grouped.to_frame()
neighbourhood = neighbourhood_grouped.to_frame()
#combine the dataframe borough and the dataframe neighbourhood into one dataframe
grouped_final = borough.merge(neighbourhood, on="Postcode")
#Confirm shape of the newly created dataframe
grouped_final.shape

(103, 2)